In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing packages 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt 


from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, balanced_accuracy_score
from sklearn.model_selection import KFold





from sklearn.linear_model import ElasticNet


from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR

import xgboost as xgb 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import r2_score

from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")


In [ ]:
#input file 

wine = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')



In [ ]:
#conducting EDA 

wine.head(5)

Values of all columns except 'quality' are continuous. 'Quality' is the only column having discrete values. 

In [ ]:
wine.info()

Dataset contains 12 columns (all except 'Quality' have continuous values), and 1599 rows. 
No null values in dataset 

In [ ]:
wine.describe()


Some cells seem to have skewed values (i.e. their distributions are not approx. normal) 
We may do outlier analysis at a later date to improve the data 

In [ ]:
#exploring target variable = quality . seems to be an imbalanced set 

ax = sns.countplot(x=wine['quality'])
fig = plt.figure(figsize = (10,10))

for p in ax.patches:
        ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+50))
        
print(wine['quality'].value_counts())

Quality seems to be an imbalanced output set - 
there are neither 'very bad'(ranked 0,1,2) wines nor 'very good'(ranked 9,10) ones
Also there are much more 'average' (ranked 5,6) wines than any other in the data set 

In fact, 82% of the output values seem to be ranked average 


In [ ]:
corr = wine.corr()
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(corr,annot=True,ax=ax)


Studying correlation wrt output variable 'Quality' -> Top correlators with quality are : alcohol, volatile acidity, sulphates, citric acid


Other observations : 
pH, density,citric acid have a high corelation with fixed acidity (approx. .67)






In [ ]:

sns.pairplot(wine)

Other variables vs Quality graphs prove inconclusive 

Checking distribution plots of various variables (diagonal axis): 

Fixed acidity is left skewed normal ,
Volatile acidity distibution is bimodal (two peaked) 
citric acid, Alcohol, Free sulfur dioxide distribution has no clear trend 

In [ ]:
#splitting test + train sets 

X=wine.drop(['quality'],axis=1)
y=wine['quality']
sc=StandardScaler()
X=sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=42)


In [ ]:
#Feature engineering using SMOTE 

from imblearn.over_sampling import SMOTE,ADASYN
X_resample,y_resample= SMOTE(sampling_strategy={3:700, 4:700, 5:700, 6:700, 7:700, 8:700}).fit_resample(X_train,y_train)



In [ ]:
#Classification Algorithms 


models=[("Logistic Regression", LogisticRegression()),
        ("Stochastic Gradient Descent", SGDClassifier()),
        ("Linear Discriminant Analysis", LinearDiscriminantAnalysis()),
        ("Decision Tree", DecisionTreeClassifier()),
        ("Random Forest", RandomForestClassifier()),
        ("Extra Trees", ExtraTreesClassifier()),
        ("Gradient Boosting", GradientBoostingClassifier()),
        ("KNeighbors", KNeighborsClassifier()),
        ("SVM", SVC()),
        ("Naive Bayes", GaussianNB()),
        ("Ada Boost", AdaBoostClassifier()),
        ("Gradient Boosted Trees",xgb.XGBClassifier())]

for name, model in models:
    skf=StratifiedKFold(n_splits=10)
    results = cross_val_score(model, X_resample, y_resample, cv=skf, scoring='accuracy')
    print(f"\x1b[96m{name}\x1b[0m: \x1b[95m{results.mean():.4f}\x1b[0m ± {results.std():.4f}")


In [ ]:
from sklearn.model_selection import GridSearchCV
from pprint import pprint
pprint(xgb.XGBClassifier().get_params())


In [ ]:
param_grid = {
    'colsample_bytree': [0.3, 0.7],
    'n_estimators': [50,100],
    'max_depth': [2, 5]
}
gbm = xgb.XGBClassifier()
skf=StratifiedKFold(n_splits=10)

grid_mse = GridSearchCV(estimator=gbm,param_grid=param_grid, scoring='accuracy', cv=skf)
grid_mse.fit(X_resample, y_resample)
print("Best parameters found: ", grid_mse.best_params_)
print(grid_mse.best_score_)

#boosted accuracy from  64 to 66%

In [ ]:
#let's analyse the params used for Random Forest 

from pprint import pprint
pprint(RandomForestClassifier().get_params())




In [ ]:
#from sklearn.inspection import permutation_importance
#rf=RandomForestClassifier(n_estimators=100)
#rf.fit(X_train,y_train)
#perm_importance = permutation_importance(RandomForestClassifier(), X_test, y_test)


In [ ]:
#hyperparameter tuning for few of these parameters 

param_grid2 = {
    'bootstrap': [True],
    'criterion': ['gini'],
    'n_estimators': [100],    
    'max_depth': [10,20],
    'max_features' : ['None','sqrt','0.6']}
    
    
rf = RandomForestClassifier()

skf=StratifiedKFold(n_splits=10)    

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid2, scoring='accuracy',
                          cv = skf) 
grid_search.fit(X_resample, y_resample)
    
print("Best parameters found: ", grid_search.best_params_)
print(grid_search.best_score_)

#accuracy from 69% to 68%   = X_train, y_train

    

